In [1]:
import cv2
import numpy as np
import time
import os

ROOT = ".\crime videos"
videos = os.listdir("./crime videos")

net = cv2.dnn.readNet("yolov3-tiny.weights","yolov3-tiny.cfg")

classes = []
with open("./coco.names","r") as f:
    classes = [line.strip() for line in f.readlines()]

In [2]:
def loadVideo(sno):
    return os.path.join(ROOT, videos[sno-1])

video = loadVideo(1)

In [3]:
layer_names = net.getLayerNames()
outputlayers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]


colors= np.random.uniform(0,255,size=(len(classes),3))

cap=cv2.VideoCapture(video)

while True:
    data = "Human Detection: False"
    ret,frame= cap.read() 
    
    if ret==True:
        height,width,channels = frame.shape
        blob = cv2.dnn.blobFromImage(frame,0.00392,(320,320),(0,0,0),True,crop=False)    

        net.setInput(blob)
        outs = net.forward(outputlayers)

        class_ids=[]
        confidences=[]
        boxes=[]

        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                
                # could modify this to meet your requirements
                if confidence > 0.5:
                    #object detected
                    center_x= int(detection[0]*width)
                    center_y= int(detection[1]*height)
                    w = int(detection[2]*width)
                    h = int(detection[3]*height)

                    #rectangle co-ordinaters
                    x=int(center_x - w/2)
                    y=int(center_y - h/2)

                    boxes.append([x,y,w,h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.4,0.6)
        
        for i in range(len(boxes)):
            if i in indexes:
                label = str(classes[class_ids[i]])
                confidence= confidences[i]
                
                if label=="person":
                    x,y,w,h = boxes[i]
                    data = "Human Detection: True"
                    
                    # Custom code to do miscellaneous tasks (like predicting anomalies) can go down here
                    
                    # Custom code ends
                
                color = colors[class_ids[i]]
                cv2.rectangle(frame,(x,y),(x+w,y+h),color,2)
                cv2.putText(frame,label+":"+str(round(confidence,2)), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX,0.4,(255,255,255),2)
                
        cv2.putText(frame, data, (0,20), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (106, 0, 0), 2)

        cv2.imshow("Image",frame)
        key = cv2.waitKey(1)

        if cv2.waitKey(1) & 0xFF==ord('q'):
            break
    
    else:
        break

        

cap.release()
cv2.destroyAllWindows()